# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1d00a58430>
├── 'a' --> tensor([[-0.1165,  0.7114, -0.0227],
│                   [-0.1095, -0.1939,  1.2864]])
└── 'x' --> <FastTreeValue 0x7f1d00a58b20>
    └── 'c' --> tensor([[-1.0578, -0.7353, -1.9373, -0.0341],
                        [ 0.7240,  0.1264, -0.9299,  0.4492],
                        [ 0.5426,  0.0891,  0.7551, -0.6982]])

In [4]:
t.a

tensor([[-0.1165,  0.7114, -0.0227],
        [-0.1095, -0.1939,  1.2864]])

In [5]:
%timeit t.a

59.7 ns ± 0.0983 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1d00a58430>
├── 'a' --> tensor([[ 0.4631, -1.1466,  0.7919],
│                   [-1.2490,  0.5655, -0.1500]])
└── 'x' --> <FastTreeValue 0x7f1d00a58b20>
    └── 'c' --> tensor([[-1.0578, -0.7353, -1.9373, -0.0341],
                        [ 0.7240,  0.1264, -0.9299,  0.4492],
                        [ 0.5426,  0.0891,  0.7551, -0.6982]])

In [7]:
%timeit t.a = new_value

58.2 ns ± 0.0396 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1165,  0.7114, -0.0227],
               [-0.1095, -0.1939,  1.2864]]),
    x: Batch(
           c: tensor([[-1.0578, -0.7353, -1.9373, -0.0341],
                      [ 0.7240,  0.1264, -0.9299,  0.4492],
                      [ 0.5426,  0.0891,  0.7551, -0.6982]]),
       ),
)

In [10]:
b.a

tensor([[-0.1165,  0.7114, -0.0227],
        [-0.1095, -0.1939,  1.2864]])

In [11]:
%timeit b.a

54.6 ns ± 0.063 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6113, -0.3718,  0.5499],
               [-0.8059,  0.3412,  1.9071]]),
    x: Batch(
           c: tensor([[-1.0578, -0.7353, -1.9373, -0.0341],
                      [ 0.7240,  0.1264, -0.9299,  0.4492],
                      [ 0.5426,  0.0891,  0.7551, -0.6982]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.155 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

836 ns ± 10.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 36.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 257 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 776 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1d00a616a0>
├── 'a' --> tensor([[[-0.1165,  0.7114, -0.0227],
│                    [-0.1095, -0.1939,  1.2864]],
│           
│                   [[-0.1165,  0.7114, -0.0227],
│                    [-0.1095, -0.1939,  1.2864]],
│           
│                   [[-0.1165,  0.7114, -0.0227],
│                    [-0.1095, -0.1939,  1.2864]],
│           
│                   [[-0.1165,  0.7114, -0.0227],
│                    [-0.1095, -0.1939,  1.2864]],
│           
│                   [[-0.1165,  0.7114, -0.0227],
│                    [-0.1095, -0.1939,  1.2864]],
│           
│                   [[-0.1165,  0.7114, -0.0227],
│                    [-0.1095, -0.1939,  1.2864]],
│           
│                   [[-0.1165,  0.7114, -0.0227],
│                    [-0.1095, -0.1939,  1.2864]],
│           
│                   [[-0.1165,  0.7114, -0.0227],
│                    [-0.1095, -0.1939,  1.2864]]])
└── 'x' --> <FastTreeValue 0x7f1c49cf3bb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1c5095bcd0>
├── 'a' --> tensor([[-0.1165,  0.7114, -0.0227],
│                   [-0.1095, -0.1939,  1.2864],
│                   [-0.1165,  0.7114, -0.0227],
│                   [-0.1095, -0.1939,  1.2864],
│                   [-0.1165,  0.7114, -0.0227],
│                   [-0.1095, -0.1939,  1.2864],
│                   [-0.1165,  0.7114, -0.0227],
│                   [-0.1095, -0.1939,  1.2864],
│                   [-0.1165,  0.7114, -0.0227],
│                   [-0.1095, -0.1939,  1.2864],
│                   [-0.1165,  0.7114, -0.0227],
│                   [-0.1095, -0.1939,  1.2864],
│                   [-0.1165,  0.7114, -0.0227],
│                   [-0.1095, -0.1939,  1.2864],
│                   [-0.1165,  0.7114, -0.0227],
│                   [-0.1095, -0.1939,  1.2864]])
└── 'x' --> <FastTreeValue 0x7f1cd81fb2b0>
    └── 'c' --> tensor([[-1.0578, -0.7353, -1.9373, -0.0341],
                        [ 0.7240,  0.1264, -0.9299,  0.4492],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 54.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.2 µs ± 71.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.0578, -0.7353, -1.9373, -0.0341],
                       [ 0.7240,  0.1264, -0.9299,  0.4492],
                       [ 0.5426,  0.0891,  0.7551, -0.6982]],
              
                      [[-1.0578, -0.7353, -1.9373, -0.0341],
                       [ 0.7240,  0.1264, -0.9299,  0.4492],
                       [ 0.5426,  0.0891,  0.7551, -0.6982]],
              
                      [[-1.0578, -0.7353, -1.9373, -0.0341],
                       [ 0.7240,  0.1264, -0.9299,  0.4492],
                       [ 0.5426,  0.0891,  0.7551, -0.6982]],
              
                      [[-1.0578, -0.7353, -1.9373, -0.0341],
                       [ 0.7240,  0.1264, -0.9299,  0.4492],
                       [ 0.5426,  0.0891,  0.7551, -0.6982]],
              
                      [[-1.0578, -0.7353, -1.9373, -0.0341],
                       [ 0.7240,  0.1264, -0.9299,  0.4492],
                       [ 0.5426,  0.0891,  0.7551, -0.6982]],

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 114 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.0578, -0.7353, -1.9373, -0.0341],
                      [ 0.7240,  0.1264, -0.9299,  0.4492],
                      [ 0.5426,  0.0891,  0.7551, -0.6982],
                      [-1.0578, -0.7353, -1.9373, -0.0341],
                      [ 0.7240,  0.1264, -0.9299,  0.4492],
                      [ 0.5426,  0.0891,  0.7551, -0.6982],
                      [-1.0578, -0.7353, -1.9373, -0.0341],
                      [ 0.7240,  0.1264, -0.9299,  0.4492],
                      [ 0.5426,  0.0891,  0.7551, -0.6982],
                      [-1.0578, -0.7353, -1.9373, -0.0341],
                      [ 0.7240,  0.1264, -0.9299,  0.4492],
                      [ 0.5426,  0.0891,  0.7551, -0.6982],
                      [-1.0578, -0.7353, -1.9373, -0.0341],
                      [ 0.7240,  0.1264, -0.9299,  0.4492],
                      [ 0.5426,  0.0891,  0.7551, -0.6982],
                      [-1.0578, -0.7353, -1.9373, -0.0341],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 589 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

314 µs ± 2.29 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
